# Як кластеризація підвищує точність моделей "з учителем" (kmeans + renadomForest)

Курс: Математичне моделювання в R

---

Увага. Для використання подібного підходу до підвищення точності прогнозів варто переконатися, що це працює.

In [1]:
# install.packages("carData")

In [2]:
library(randomForest)
library(cluster)
library(modelr)
library(dplyr)

randomForest 4.7-1.1

Type rfNews() to see new features/changes/bug fixes.


Attaching package: 'dplyr'


The following object is masked from 'package:randomForest':

    combine


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [3]:
Sys.setlocale("LC_CTYPE", "ukrainian")
options(warn = -1)

[1] "Ukrainian_Ukraine.1251"

## Набір даних

Детальна інформація про набір даних вже описана у матеріалі "Дерева рішень. Регресія. Баланс кредитної карти" (Примітка. Додати лінк)

In [4]:
data <- read.csv("data/credit_card_balance.csv")

Переглянемо структуру даних:

In [5]:
str(data) 

'data.frame':	400 obs. of  12 variables:
 $ X        : int  1 2 3 4 5 6 7 8 9 10 ...
 $ Income   : num  14.9 106 104.6 148.9 55.9 ...
 $ Limit    : int  3606 6645 7075 9504 4897 8047 3388 7114 3300 6819 ...
 $ Rating   : int  283 483 514 681 357 569 259 512 266 491 ...
 $ Cards    : int  2 3 4 3 2 4 2 2 5 3 ...
 $ Age      : int  34 82 71 36 68 77 37 87 66 41 ...
 $ Education: int  11 15 11 11 16 10 12 9 13 19 ...
 $ Gender   : chr  "Male" "Female" "Male" "Female" ...
 $ Student  : chr  "No" "Yes" "No" "No" ...
 $ Married  : chr  "Yes" "Yes" "No" "No" ...
 $ Ethnicity: chr  "Caucasian" "Asian" "Asian" "Asian" ...
 $ Balance  : int  333 903 580 964 331 1151 203 872 279 1350 ...


Підготуємо дані до моделювання. Перетворимо категоріальні показники до факторів:

In [6]:
data$X <- NULL
data$Gender <-  factor(data$Gender)
data$Student <- factor(data$Student)
data$Married <- factor(data$Married)
data$Ethnicity <- factor(data$Ethnicity)

---

## Тренувальна та тестова вибірки

Розділимо на тестову та тренувальну вибірки:

In [7]:
set.seed(2) 
train_index <- sample(nrow(data), size = 0.5*nrow(data))
train_data <- data[train_index, ]
test_data <- data[-train_index, ]
nrow(train_data)
nrow(test_data)

[1] 200

[1] 200

---

## Побудова моделі на основі RandomForest

In [8]:
rf_m1 <-randomForest(Balance~.,data=train_data)
summary(rf_m1)

                Length Class  Mode     
call              3    -none- call     
type              1    -none- character
predicted       200    -none- numeric  
mse             500    -none- numeric  
rsq             500    -none- numeric  
oob.times       200    -none- numeric  
importance       10    -none- numeric  
importanceSD      0    -none- NULL     
localImportance   0    -none- NULL     
proximity         0    -none- NULL     
ntree             1    -none- numeric  
mtry              1    -none- numeric  
forest           11    -none- list     
coefs             0    -none- NULL     
y               200    -none- numeric  
test              0    -none- NULL     
inbag             0    -none- NULL     
terms             3    terms  call     

In [9]:
test_prediction1 <- round(predict(object=rf_m1, test_data))
head(test_prediction1)

2    4    5    6    7   10 
1038 1331  433 1010  182 1224

Перевіримо детермінацію та похибки

In [10]:
rsquare(rf_m1, data = train_data)
rsquare(rf_m1, data = test_data)

[1] 0.97384

[1] 0.8831837

In [11]:
rmse(rf_m1, data = train_data)
rmse(rf_m1, data = test_data)

[1] 76.96301

[1] 150.6356

---

## Кластеризуємо числові дані

Для початку об'єднаємо усі дані в один датасет, памятаємо, що пропорція розбиття `280/120` значень:

In [12]:
data <- train_data |> bind_rows(test_data)

Згенеруємо, наприклад, 5 кластерів длише для числових показників без параметру `Balance` (чому без балансу?):

In [13]:
data_k <- data |>
    select(-Balance) |>
    select_if(is.numeric)
head(data_k)

,Income,Limit,Rating,Cards,Age,Education
,<dbl>,<int>,<int>,<int>,<int>,<int>
1,27.794,3807,301,4,35,8
2,50.699,3977,304,2,84,17
3,180.379,9310,665,3,67,8
4,73.327,6555,472,2,43,15
5,30.413,3690,299,2,25,15
6,13.433,1134,112,3,70,14


In [14]:
set.seed(2)
clusters <- kmeans(data_k, 4)
clusters$cluster

[1] 3 3 1 2 3 4 2 2 3 3 3 3 4 2 3 4 4 3 3 3 3 4 4 2 4 4 3 2 3 3 3 3 4 3 4 2 2
 [38] 2 3 3 3 2 3 2 4 2 2 3 4 3 1 4 2 2 4 4 4 1 4 3 3 3 4 3 2 2 4 3 3 3 3 2 4 3
 [75] 4 4 3 3 2 4 4 4 1 4 3 4 3 1 3 4 3 3 3 3 3 4 4 3 3 2 3 2 3 2 4 2 2 4 2 1 3
[112] 4 3 4 2 4 4 4 3 1 3 3 3 3 3 2 4 2 3 4 4 4 2 3 3 4 4 1 3 2 4 4 4 2 4 3 4 3
[149] 3 4 3 4 4 4 1 4 4 4 2 4 3 4 4 2 3 4 2 3 3 3 2 4 2 3 4 4 4 2 3 3 2 2 4 3 2
[186] 4 1 4 1 4 4 3 2 3 4 3 3 1 2 4 2 1 3 2 4 2 4 3 2 2 4 2 3 3 4 3 3 4 2 3 4 4
[223] 2 3 3 4 3 4 3 3 4 3 3 4 4 3 3 3 3 3 3 4 4 2 2 4 3 4 3 1 3 4 2 2 4 3 4 4 4
[260] 4 2 2 4 4 4 4 3 1 3 4 3 3 3 3 4 3 4 2 4 4 2 3 3 3 2 3 4 3 1 4 3 2 4 3 4 3
[297] 1 4 3 3 3 4 3 2 2 2 3 3 2 4 3 4 4 4 4 4 4 4 3 2 3 4 4 3 4 3 3 3 4 3 2 4 4
[334] 2 2 4 4 4 4 3 1 4 4 3 4 3 2 3 3 2 3 3 2 3 2 3 4 4 4 4 1 4 3 3 3 3 3 4 1 3
[371] 3 4 3 1 4 3 2 3 3 1 4 3 3 3 2 2 3 4 3 3 3 3 4 3 4 3 2 3 4 3

Додамо кластери як фактори до даних:

In [15]:
data <- data |>
    mutate(cluster = clusters$cluster)

Розібємо знову на тестову та тренувальну вибірки:

In [16]:
train_data <- data[1:200,]
test_data <- data[201:400,]

In [17]:
rf_m2 <-randomForest(Balance~.,data=train_data)
summary(rf_m2)

                Length Class  Mode     
call              3    -none- call     
type              1    -none- character
predicted       200    -none- numeric  
mse             500    -none- numeric  
rsq             500    -none- numeric  
oob.times       200    -none- numeric  
importance       11    -none- numeric  
importanceSD      0    -none- NULL     
localImportance   0    -none- NULL     
proximity         0    -none- NULL     
ntree             1    -none- numeric  
mtry              1    -none- numeric  
forest           11    -none- list     
coefs             0    -none- NULL     
y               200    -none- numeric  
test              0    -none- NULL     
inbag             0    -none- NULL     
terms             3    terms  call     

In [18]:
test_prediction2 <- round(predict(object=rf_m2, test_data))
head(test_prediction2)

201  202  203  204  205  206 
1026 1352  423 1036  173 1211

In [19]:
rsquare(rf_m1, data = train_data)
rsquare(rf_m1, data = test_data)

[1] 0.97384

[1] 0.8831837

In [20]:
rsquare(rf_m2, data = train_data)
rsquare(rf_m2, data = test_data)

[1] 0.9746383

[1] 0.8927143

In [21]:
rmse(rf_m1, data = train_data)
rmse(rf_m1, data = test_data)

[1] 76.96301

[1] 150.6356

In [22]:
rmse(rf_m2, data = train_data)
rmse(rf_m2, data = test_data)

[1] 75.77792

[1] 144.5014